#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
! pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18917 sha256=2e0ba3f3d3a70871b8ca85257e2bce655a46f41564ec97f59c323d2bde6bbf41
  Stored in directory: /tmp/pip-ephem-wheel-cache-7bxk6jhq/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-10-17 23:54:01.071100: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 23:54:01.104873: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 23:54:01.105591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-17 23:54:01.607738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load extra features

In [5]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [6]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-17
tomorrows_datestring:  2023-10-18


In [7]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'
#json_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20230925_143704_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231003_031227_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231011_223042_Boston311KerasNLP.h5'
#json_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.json'
#model_file = './daily_models/Boston311KerasNLP/20231011_232344_Boston311KerasNLP.h5'

json_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.json'
model_file = './daily_models/Boston311KerasNLP/20231012_005720_Boston311KerasNLP.h5'
#json_file = "./daily_models/Boston311KerasNLP/20231014_234834_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231014_234834_Boston311KerasNLP.h5"
#json_file = "./daily_models/Boston311KerasNLP/20231015_004906_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231015_004906_Boston311KerasNLP.h5"

#json_file = "./daily_models/Boston311KerasNLP/20231016_113838_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231016_113838_Boston311KerasNLP.h5"

#json_file = "./daily_models/Boston311KerasNLP/20231016_121447_Boston311KerasNLP.json"
#model_file = "./daily_models/Boston311KerasNLP/20231016_121447_Boston311KerasNLP.h5"

json_file = "./daily_models/Boston311KerasNLP/20231016_132122_Boston311KerasNLP.json"
model_file = "./daily_models/Boston311KerasNLP/20231016_132122_Boston311KerasNLP.h5"

kerasNLP_model = Boston311KerasNLP()
kerasNLP_model.load( json_file, model_file)
kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


In [8]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [9]:
data = kerasNLP_model.load_data( data=mydata, train_or_predict='predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [10]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,geom_4326,source,survival_time,event,ward_number,survival_time_hours
2723524,101005112423,2023-10-15 19:34:24,2023-10-17 04:30:00,NaT,ONTIME,Open,,Request for Pothole Repair,Public Works Department,Highway Maintenance,...,50 Meadowbank Ave,2126.0,42.269630,-71.082891,0101000020E610000075A3CB144EC551C07E86B3388322...,Citizens Connect App,NaT,0,18,NaN
2723523,101005112422,2023-10-15 19:31:22,NaN,NaT,ONTIME,Open,,Schedule Bulk Item Pickup,Public Works Department,Sanitation,...,25 Wyman St,2130.0,42.321580,-71.106381,0101000020E6100000424ABBF0CEC651C0401F1A852929...,Self Service,NaT,0,10,NaN
2723522,101005112358,2023-10-15 19:12:44,2023-10-23 04:30:00,NaT,ONTIME,Open,,Ground Maintenance,Parks & Recreation Department,Park Maintenance & Safety,...,1535 Washington St,2118.0,42.339516,-71.073388,0101000020E61000005796BE63B2C451C038C57641752B...,Citizens Connect App,NaT,0,9,NaN
2723521,101005112356,2023-10-15 19:05:09,2023-10-17 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,23 Cufflin St,2135.0,42.350600,-71.174591,0101000020E6100000D4D2937E2CCB51C0A0773873E02C...,Citizens Connect App,NaT,0,22,NaN
2723520,101005112313,2023-10-15 18:55:00,2023-10-17 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,192 Boston St,2125.0,42.323160,-71.060281,0101000020E61000001DFDDDA2DBC351C049EF944A5D29...,Constituent Call,NaT,0,7,NaN
2723519,101005112297,2023-10-15 18:52:28,2023-10-17 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,158 Marine Rd,2127.0,42.330777,-71.032828,0101000020E6100000CA39A8DC19C251C0B72785E2562A...,Citizens Connect App,NaT,0,7,NaN
2723518,101005112285,2023-10-15 18:50:00,NaN,NaT,ONTIME,Open,,Mattress Pickup,Public Works Department,Sanitation,...,199 Condor St,2128.0,42.382488,-71.033051,0101000020E6100000C14092831DC251C0E2ED2A5EF530...,Constituent Call,NaT,0,1,NaN
2723517,101005112272,2023-10-15 18:48:17,NaN,NaT,ONTIME,Open,,Mattress Pickup,Public Works Department,Sanitation,...,197 Condor St,2128.0,42.382309,-71.033090,0101000020E6100000CDF5EE261EC251C035606A84EF30...,Constituent Call,NaT,0,1,NaN
2723516,101005112266,2023-10-15 18:44:49,2023-10-17 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,1748 Columbia Rd,2127.0,42.330469,-71.032580,0101000020E6100000293E51CC15C251C05F25FDD24C2A...,Citizens Connect App,NaT,0,7,NaN
2723515,101005112264,2023-10-15 18:44:09,2023-10-17 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,76 Falcon St,2128.0,42.382087,-71.036396,0101000020E6100000A263555154C251C0C2492D3EE830...,Citizens Connect App,NaT,0,1,NaN


In [11]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [12]:
clean_data.head()

,case_enquiry_id,queue_ANML02_LostFound,queue_ANML_General,queue_BHA_General,queue_BPD_Administrative,queue_BPD_Districts,queue_BPS_Administrative,queue_BPS_Transportation Administration,queue_BTDT_AVRS Interface Queue,queue_BTDT_Abandoned Bicycle,...,department_ECON,department_GEN_,department_GRNi,department_INFO,department_ISD,department_No Q,department_ONS_,department_PARK,department_PROP,department_PWDx
0,101005056209,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,101005056210,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2,101005056211,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,101005056212,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,101005056213,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
mydata = clean_data

In [14]:
mydata['case_enquiry_id']

0        101005056209
1        101005056210
2        101005056211
3        101005056212
4        101005056213
             ...     
25272    101005112313
25273    101005112356
25274    101005112358
25275    101005112422
25276    101005112423
Name: case_enquiry_id, Length: 25277, dtype: object

In [15]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        X[col] = X[col].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [16]:
#concatenate the two dataframes and reindex
df = X

In [17]:
df.shape

(270887, 7)

In [18]:


# Assuming df is your DataFrame and it has columns 'desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding'
desc_cls_embedding_flattened = np.stack(df['desc_cls_embedding'].to_numpy())
desc_pooled_embedding_flattened = np.stack(df['desc_pooled_embedding'].to_numpy())
#do the same for the rest
name_cls_embedding_flattened = np.stack(df['name_cls_embedding'].to_numpy())
name_pooled_embedding_flattened = np.stack(df['name_pooled_embedding'].to_numpy())
code_cls_embedding_flattened = np.stack(df['code_cls_embedding'].to_numpy())
code_pooled_embedding_flattened = np.stack(df['code_pooled_embedding'].to_numpy())


# Remove the old columns
df.drop(['desc_cls_embedding', 'desc_pooled_embedding'], axis=1, inplace=True)
#do the same for the rest
df.drop(['name_cls_embedding', 'name_pooled_embedding'], axis=1, inplace=True)
df.drop(['code_cls_embedding', 'code_pooled_embedding'], axis=1, inplace=True)


# Add the new flattened columns
df_desc_cls = pd.DataFrame(desc_cls_embedding_flattened, columns=[f'desc_cls_{i}' for i in range(desc_cls_embedding_flattened.shape[1])])

df_desc_pooled = pd.DataFrame(desc_pooled_embedding_flattened, columns=[f'desc_pooled_{i}' for i in range(desc_pooled_embedding_flattened.shape[1])])
#do the same for the rest
df_name_cls = pd.DataFrame(name_cls_embedding_flattened, columns=[f'name_cls_{i}' for i in range(name_cls_embedding_flattened.shape[1])])
df_name_pooled = pd.DataFrame(name_pooled_embedding_flattened, columns=[f'name_pooled_{i}' for i in range(name_pooled_embedding_flattened.shape[1])])
df_code_cls = pd.DataFrame(code_cls_embedding_flattened, columns=[f'code_cls_{i}' for i in range(code_cls_embedding_flattened.shape[1])])
df_code_pooled = pd.DataFrame(code_pooled_embedding_flattened, columns=[f'code_pooled_{i}' for i in range(code_pooled_embedding_flattened.shape[1])])



df = pd.concat([df, df_desc_cls, df_desc_pooled, df_name_cls, df_name_pooled, df_code_cls, df_code_pooled], axis=1)

In [19]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [20]:
df = df[is_numeric]

In [21]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [22]:
df.shape

(270887, 769)

In [23]:
mydata.shape

(25277, 244)

In [24]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [25]:
new_mydata.shape

(16755, 1012)

In [26]:

df = new_mydata

In [27]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [28]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
code_pooled_123             float64
code_pooled_124             float64
code_pooled_125             float64
code_pooled_126             float64
code_pooled_127             float64
Length: 1012, dtype: object


In [29]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [30]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [31]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

173/524 [========>.....................] - ETA: 0s

524/524 [==============================] - 1s 1ms/step


In [32]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.0074115763 0.0075312313 0.006558172 0.0056856554 0.0044748853 0.003209142 0.0024732822 0.0029631732 0.002783216 0.0024955869 0.0015848437 0.0013698614 0.0015505874 0.0015494677 0.0019226712 0.0018922954 0.0021640598 0.0014633301 0.0013371349 0.0020982448 0.0016141027 0.002191352 0.0018902248 0.0024502734 0.0015364994 0.0015118755 0.0013828442 0.0015772143 0.0016039346 0.0015858349 0.0019152194 0.0038218691 0.0060475194 0.004728616 0.007018377 0.0039016388 0.0047212127 0.0035283903 0.0062187086 0.0073214704 0.0059566125 0.0045704916 0.0026605304 0.0023946671 0.0043884125 0.0033095542 0.004762682 0.0039071757 0.006407129 0.0052055125 0.006020003 0.004083236 0.0032249715 0.0055654156 0.005596258 0.004784561 0.004733356 0.0030682883 0.003143382 0.0047500446 0.0024504089 0.003926873 0.0031525476 0.005049093 0.007997642 0.0020698023 0.0045525776 0.0036813456 0.0037080345 0.0040039937 0.004755035 0.0037932356 0.0017332647 0.0029395877 0.004542798 0.0037428592 0.003535307 0.0025116869 0.00

In [33]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [34]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [35]:
len(predictions_df)

16755

In [36]:
print(predictions)

[[7.4115763e-03 7.5312313e-03 6.5581719e-03 ... 3.7962829e-03
  3.4240868e-03 4.5820180e-01]
 [2.0154398e-02 1.3138226e-02 1.0371000e-02 ... 2.8900073e-03
  2.8427988e-03 5.5718839e-01]
 [4.1348074e-02 7.2919212e-02 6.5617323e-02 ... 2.1824107e-04
  3.3386025e-04 1.3243957e-02]
 ...
 [6.6519147e-01 1.0392980e-01 8.8128299e-02 ... 7.9520440e-08
  1.6335881e-06 5.2302574e-05]
 [7.1473318e-01 9.0537824e-02 7.6027609e-02 ... 1.1778681e-07
  2.0314978e-06 1.1254865e-04]
 [5.5720001e-01 1.6453116e-01 1.0096454e-01 ... 1.7109158e-05
  4.7062553e-05 2.6461205e-03]]


In [37]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [38]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20231016_132122_Boston311KerasNLP  Boston311KerasNLP    2023-10-16


In [39]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event'], axis=1).copy()

In [40]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [41]:
all_model_predictions['ml_model_name'] = model_name

In [42]:
all_model_predictions['prediction_date'] = today_datestring

In [43]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [44]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [45]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-18_00-01-05_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-18_00-01-05_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-18_00-01-05_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Connection to 195.179.236.61 closed by remote host.
lost connection


Executing: scp -P 65002 2023-10-18_00-01-05_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-18_00-01-05_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-18_00-01-05_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-18_00-01-05_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-18_00-01-05_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders


65002

In [46]:
# %%
if copy_to_prod:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    !ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-18_00-01-05_manifest.txt

Total records to process: 39934

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-18_00-01-05_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 25177.
Total records remaining: 39834.
Time for last 100 records: 0.03 seconds.
Estimated time remaining for this file: 6 seconds.
Estimated time for all files: 10 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 25077.
Total records remaining: 39734.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 second.
Estimated time for all files: 2 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 24977.
Total records remaining: 39634.
Time for last 100 records: 0 seconds.
Estimated time remaining for this file: 1 secon